# Análise de Ordens WDO
Análise detalhada das ordens geradas pelo modelo otimizado.

In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

PROJECT_ROOT = Path(os.path.dirname(os.getcwd()))
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

plt.style.use('default')
pd.set_option('display.max_columns', None)

In [2]:
# Carrega modelo otimizado
from src.data.loaders.market_data_loader import MarketDataLoader
from src.ml.features.feature_engineering import FeatureEngineering
from src.agents.market_agent import RiskManagementAgent, MarketRegimeAgent
from src.agents.agent_coordinator import AgentCoordinator

# Parâmetros otimizados
risk_agent = RiskManagementAgent(
    max_drawdown=-0.002793,
    max_position_size=2
)

regime_agent = MarketRegimeAgent(
    window_size=25
)

coordinator = AgentCoordinator([risk_agent, regime_agent])

In [3]:
# Função para análise de ordens
def analyze_orders(signals, data):
    orders = []
    position = 0
    entry_price = 0
    entry_time = None
    
    for idx, row in data.iterrows():
        signal = signals[idx]
        
        # Fecha posição existente
        if position != 0 and signal != position:
            pnl = (row['close'] - entry_price) * position
            duration = (idx - entry_time).total_seconds() / 3600  # em horas
            
            orders.append({
                'entry_time': entry_time,
                'exit_time': idx,
                'duration': duration,
                'entry_price': entry_price,
                'exit_price': row['close'],
                'direction': 'LONG' if position > 0 else 'SHORT',
                'pnl': pnl,
                'pnl_percent': (pnl / entry_price) * 100
            })
            position = 0
        
        # Abre nova posição
        if position == 0 and signal != 0:
            position = signal
            entry_price = row['close']
            entry_time = idx
    
    return pd.DataFrame(orders)

In [4]:
def analyze_orders(signals, data):
    orders = []
    position = 0
    entry_price = 0
    entry_time = None
    
    # Convertendo sinais para Series com mesmo índice do data
    signals = pd.Series(signals, index=data.index)
    
    for idx, row in data.iterrows():
        signal = signals.loc[idx]
        
        # Fecha posição existente
        if position != 0 and signal != position:
            pnl = (row['close'] - entry_price) * position
            duration = (idx - entry_time).total_seconds() / 3600
            
            orders.append({
                'entry_time': entry_time,
                'exit_time': idx,
                'duration': duration,
                'entry_price': entry_price,
                'exit_price': row['close'],
                'direction': 'LONG' if position > 0 else 'SHORT',
                'pnl': pnl,
                'pnl_percent': (pnl / entry_price) * 100
            })
            position = 0
        
        # Abre nova posição
        if position == 0 and signal != 0:
            position = signal
            entry_price = row['close']
            entry_time = idx
    
    return pd.DataFrame(orders)

In [5]:
def get_signals(data, risk_agent, regime_agent):
    signals = np.zeros(len(data))
    position = 0
    
    for i in range(1, len(data)):
        if data['rsi'].iloc[i] < 30 and position <= 0:
            signals[i] = 1
        elif data['rsi'].iloc[i] > 70 and position >= 0:
            signals[i] = -1
            
        if position != 0:
            ret = (data['close'].iloc[i] - data['close'].iloc[i-1]) / data['close'].iloc[i-1]
            if abs(ret) > abs(risk_agent.max_drawdown):
                signals[i] = 0
                
        position = signals[i]
    
    return signals

In [6]:
# Gera e analisa ordens
signals = get_signals(features, risk_agent, regime_agent)
orders_df = analyze_orders(signals, data)

print('\nEstatísticas das Ordens:')
print(f'Total de trades: {len(orders_df)}')
print(f'Win rate: {(orders_df.pnl > 0).mean():.2%}')
print(f'Média duração (horas): {orders_df.duration.mean():.2f}')
print(f'PnL médio: R${orders_df.pnl.mean():.2f}')
print(f'PnL total: R${orders_df.pnl.sum():.2f}')

# Análise por direção
direction_stats = orders_df.groupby('direction').agg({
    'pnl': ['count', 'mean', 'sum'],
    'duration': 'mean',
    'pnl_percent': 'mean'
}).round(2)

print('\nEstatísticas por Direção:')
print(direction_stats)

NameError: name 'features' is not defined

In [ ]:
# Visualizações
plt.figure(figsize=(15, 5))

# PnL acumulado
plt.subplot(1, 2, 1)
plt.plot(orders_df.pnl.cumsum())
plt.title('PnL Acumulado')
plt.grid(True)

# Distribuição de retornos
plt.subplot(1, 2, 2)
sns.histplot(orders_df.pnl_percent, bins=50)
plt.title('Distribuição dos Retornos (%)')
plt.grid(True)

plt.tight_layout()
plt.show()